# Fetch Token Pools addr 

(network slug, base asset addr)

In [74]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get API key
api_key = os.getenv('API_KEY')

# Set request headers
headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': api_key,
}

def fetch_spot_pairs(network_slug, base_asset_contract_address):
    results = []
    
    url = f'https://pro-api.coinmarketcap.com/v4/dex/spot-pairs/latest?network_slug={network_slug}&base_asset_contract_address={base_asset_contract_address}'
    
    # Send request
    response = requests.get(url, headers=headers)
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        results = data['data']  # Get data
    else:
        print(f"Error for {network_slug} and {base_asset_contract_address}: {response.status_code} - {response.text}")
    
    return results

def save_to_csv(spot_pairs_data, base_asset_name, network_slug):
    # Create folder
    folder_name = 'tokens_pool_addr'
    os.makedirs(folder_name, exist_ok=True)

    # Extract needed fields and save
    rows = []
    for item in spot_pairs_data:
        for quote in item['quote']:
            row = {
                'contract_address': item['contract_address'],
                'name': item['name'],
                'base_asset_id': item['base_asset_id'],
                'base_asset_name': item['base_asset_name'],
                'base_asset_symbol': item['base_asset_symbol'],
                'base_asset_contract_address': item['base_asset_contract_address'],
                'quote_asset_id': item['quote_asset_id'],
                'quote_asset_name': item['quote_asset_name'],
                'quote_asset_symbol': item['quote_asset_symbol'],
                'quote_asset_contract_address': item['quote_asset_contract_address'],
                'dex_id': item['dex_id'],
                'dex_slug': item['dex_slug'],
                'network_id': item['network_id'],
                'network_slug': item['network_slug'],
                'last_updated': item['last_updated'],
                'created_at': item['created_at'],
                'price': quote['price'],
                'volume_24h': quote['volume_24h'],
                'percent_change_price_1h': quote['percent_change_price_1h'],
                'percent_change_price_24h': quote['percent_change_price_24h'],
                'liquidity': quote['liquidity'],
                'fully_diluted_value': quote['fully_diluted_value'],
            }
            rows.append(row)

    # Convert data to DataFrame
    df = pd.DataFrame(rows)

    # Save to CSV file
    file_name = f"{base_asset_name}_{network_slug}.csv"
    file_path = os.path.join(folder_name, file_name)
    df.to_csv(file_path, index=False)
    print(f"Saved {file_path}")  # Output saved file path

# Example usage
network_slugs = ['Solana', 'Ethereum']  # List of networks
base_asset_contract_addresses = [
    'HwdwHMkaTkYREmWwQDGNgbtAou43nE9ACgdxkW2Wpump',  # Specific token address for Solana
    '2zMMhcVQEXDtdE6vsFS7S7D5oUodfJHE8vd1gnBouauv'
]

for network_slug in network_slugs:
    for base_asset_contract_address in base_asset_contract_addresses:
        spot_pairs_data = fetch_spot_pairs(network_slug, base_asset_contract_address)
        if spot_pairs_data:
            base_asset_name = spot_pairs_data[0]['base_asset_name']  # Get base asset name
            save_to_csv(spot_pairs_data, base_asset_name, network_slug)
        else:
            print(f"No data retrieved for {network_slug} and {base_asset_contract_address}")

Saved tokens_pool_addr/NYE_Solana.csv
Saved tokens_pool_addr/Pudgy Penguins_Solana.csv
No data retrieved for Ethereum and HwdwHMkaTkYREmWwQDGNgbtAou43nE9ACgdxkW2Wpump
No data retrieved for Ethereum and 2zMMhcVQEXDtdE6vsFS7S7D5oUodfJHE8vd1gnBouauv


# Read Pool Addr 

In [204]:
import os
import pandas as pd

def load_contract_pairs_from_csv(folder_name):
    contract_pairs = []

    # 遍历指定文件夹中的所有 CSV 文件
    for file_name in os.listdir(folder_name):
        if file_name.endswith('.csv'):
            # 获取基础资产名称和网络 slug
            base_asset_name, network_slug = os.path.splitext(file_name)[0].split('_')
            file_path = os.path.join(folder_name, file_name)

            # 读取 CSV 文件
            df = pd.read_csv(file_path)

            # 检查 DataFrame 是否为空
            if not df.empty:
                # 遍历 DataFrame 的每一行
                for index, row in df.iterrows():
                    pool_contract_address = row['contract_address']
                    pool_created_time=row['created_at']
                    contract_pairs.append((pool_contract_address, network_slug, base_asset_name,pool_created_time))

    return contract_pairs

# 示例调用
folder_name = 'tokens_pool_addr'
contract_pairs = load_contract_pairs_from_csv(folder_name)

# 输出结果
for pair in contract_pairs:
    print(pair)



('B4Vwozy1FGtp8SELXSXydWSzavPUGnJ77DURV2k4MhUV', 'Solana', 'Pudgy Penguins', '2024-12-17T13:00:30.000Z')
('HQCtRaHWyfqGcS7gFmvAn2vVgG7ZT2PQjBdsPd4RGNZy', 'Solana', 'Pudgy Penguins', '2024-12-17T14:22:52.000Z')
('2ZxdV1nxxUNrNey4UtXdsRGh2yYKJu1K8tUYT6qg97o3', 'Solana', 'Pudgy Penguins', '2024-12-17T13:20:11.000Z')
('A3N64gxHQL8b2cQ75JM3nVqoRFYWmntaEUFfDmhYe1Nv', 'Solana', 'Pudgy Penguins', '2024-12-17T13:00:31.000Z')
('FAqh648xeeaTqL7du49sztp9nfj5PjRQrfvaMccyd9cz', 'Solana', 'Pudgy Penguins', '2024-12-17T13:00:51.000Z')
('BR3EY6tFKwKci18hMaE4qAD4LWBhBTiRuHb6JqenxczT', 'Solana', 'Pudgy Penguins', '2024-12-17T13:29:31.000Z')
('GsTT2Jqv5EXr3vAmLcDyeY4DYDKDS1SRyzjWdZVm3pYb', 'Solana', 'Pudgy Penguins', '2024-12-17T12:46:40.000Z')
('4r2AGQRjuLpGL9o1vuW1mkcVHvE4DTi7YZJhqGHj2qeR', 'Solana', 'Pudgy Penguins', '2024-12-17T12:46:21.000Z')
('A232efjrg4Bf95skJhy9EhqNMpM1Lz3G6iBNr6NoQfb7', 'Solana', 'Pudgy Penguins', '2024-12-17T12:57:22.000Z')
('8GsWExrRFeBj1Nh8gCuVQZo3f6yzd1YFofGbwebh7uFh', 'Solan

# Get OHLCV 

In [207]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv

# 加载 .env 文件中的环境变量
load_dotenv()

# 获取 API 密钥
api_key = os.getenv('API_KEY')

# 设置请求头
headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': api_key,
}

def fetch_ohlcv_data(contract_address, network_slug):
    url = f'https://pro-api.coinmarketcap.com/v4/dex/pairs/ohlcv/historical?contract_address={contract_address}&network_slug={network_slug}&time_period=hourly&interval=1h&count=10'
    # 发送请求
    response = requests.get(url, headers=headers)

    # 检查请求是否成功
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching data for {contract_address} on network {network_slug}: {response.status_code} - {response.text}")
        return None

def save_ohlcv_to_csv(data, contract_address, base_asset_name, network_slug):
    # 创建主文件夹
    folder_name = 'tokens_pool_addr_ohlcv'
    os.makedirs(folder_name, exist_ok=True)

    # 创建子文件夹
    sub_folder_name = os.path.join(folder_name, f"{base_asset_name}_{network_slug}")
    os.makedirs(sub_folder_name, exist_ok=True)

    if data and 'data' in data:
        if not data['data']:
            print(f"No data found for {base_asset_name} on {network_slug}. Skipping...")
            return  # 如果没有数据，返回

        quotes = data['data'][0]['quotes']
        dataframes = []

        for quote in quotes:
            quote_data = quote['quote'][0]  # 取出每个 quote 的数据
            quote_data['time_open'] = quote['time_open']
            quote_data['time_close'] = quote['time_close']
            dataframes.append(pd.DataFrame([quote_data]))  # 将每个 quote 转换为 DataFrame 并添加到列表

        # 合并所有 DataFrame
        df = pd.concat(dataframes, ignore_index=True)

        # 保存为 CSV 文件，以合约地址命名
        file_name = f"{contract_address}.csv"
        file_path = os.path.join(sub_folder_name, file_name)
        df.to_csv(file_path, index=False)
        print(f"Saved {file_path}")  # 输出保存的文件路径
    else:
        print(f"Invalid data format for {base_asset_name} on {network_slug}. Skipping...")

def fetch_and_save_multiple_contracts(contract_pairs):
    for contract_address, network_slug, base_asset_name, pool_created_time in contract_pairs:
        # 获取 OHLCV 数据
        data = fetch_ohlcv_data(contract_address, network_slug)
        if data and 'data' in data and data['data']:
            # 保存数据到 CSV
            save_ohlcv_to_csv(data, contract_address, base_asset_name, network_slug)

# 执行函数
fetch_and_save_multiple_contracts(contract_pairs)

Saved tokens_pool_addr_ohlcv/Pudgy Penguins_Solana/B4Vwozy1FGtp8SELXSXydWSzavPUGnJ77DURV2k4MhUV.csv
Saved tokens_pool_addr_ohlcv/Pudgy Penguins_Solana/HQCtRaHWyfqGcS7gFmvAn2vVgG7ZT2PQjBdsPd4RGNZy.csv
Saved tokens_pool_addr_ohlcv/Pudgy Penguins_Solana/2ZxdV1nxxUNrNey4UtXdsRGh2yYKJu1K8tUYT6qg97o3.csv
Saved tokens_pool_addr_ohlcv/Pudgy Penguins_Solana/A3N64gxHQL8b2cQ75JM3nVqoRFYWmntaEUFfDmhYe1Nv.csv
Saved tokens_pool_addr_ohlcv/Pudgy Penguins_Solana/FAqh648xeeaTqL7du49sztp9nfj5PjRQrfvaMccyd9cz.csv
Saved tokens_pool_addr_ohlcv/Pudgy Penguins_Solana/BR3EY6tFKwKci18hMaE4qAD4LWBhBTiRuHb6JqenxczT.csv
Saved tokens_pool_addr_ohlcv/Pudgy Penguins_Solana/GsTT2Jqv5EXr3vAmLcDyeY4DYDKDS1SRyzjWdZVm3pYb.csv
Saved tokens_pool_addr_ohlcv/Pudgy Penguins_Solana/4r2AGQRjuLpGL9o1vuW1mkcVHvE4DTi7YZJhqGHj2qeR.csv
Saved tokens_pool_addr_ohlcv/Pudgy Penguins_Solana/A232efjrg4Bf95skJhy9EhqNMpM1Lz3G6iBNr6NoQfb7.csv
Saved tokens_pool_addr_ohlcv/Pudgy Penguins_Solana/8GsWExrRFeBj1Nh8gCuVQZo3f6yzd1YFofGbwebh7uFh.csv


# OHLCV according to start-time

In [1]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv
from datetime import datetime, timedelta, timezone

# 加载环境变量
load_dotenv()

# 从环境变量中获取 API 密钥
api_key = os.getenv('API_KEY')
headers = {
    'X-CMC_PRO_API_KEY': api_key,
    'Accepts': 'application/json'
}

def fetch_ohlcv_data(contract_address, network_slug, time_start):
    url = f'https://pro-api.coinmarketcap.com/v4/dex/pairs/ohlcv/historical?contract_address={contract_address}&network_slug={network_slug}&time_period=hourly&interval=1h&count=50&time_start={time_start}'
    
    # 发送请求
    print(f"Fetching data from URL: {url}")  # 调试信息
    response = requests.get(url, headers=headers)

    # 检查请求是否成功
    if response.status_code == 200:
        print("Request successful.")  # 调试信息
        return response.json()
    else:
        print(f"Error fetching data for {contract_address} on network {network_slug}: {response.status_code} - {response.text}")
        return None

def main(contract_address, network_slug, time_start):
    while True:
        print(f"Starting fetch for time_start: {time_start}")  # 调试信息
        
        # 获取数据
        data = fetch_ohlcv_data(contract_address, network_slug, time_start)
        
        if data:
            print("Data fetched successfully.")  # 调试信息
            if 'data' in data and len(data['data']) > 0:
                quotes = data['data'][0]['quotes']  # 更新为你提供的格式
                print(f"Number of quotes received: {len(quotes)}")  # 调试信息
                dataframes = []  # 每次循环重置数据帧列表
                for quote in quotes:
                    quote_data = quote['quote'][0]  # 取出每个 quote 的数据
                    quote_data['time_open'] = quote['time_open']
                    quote_data['time_close'] = quote['time_close']
                    dataframes.append(pd.DataFrame([quote_data]))  # 将每个 quote 转换为 DataFrame 并添加到列表
                    print(f"Processed quote: {quote_data}")  # 调试信息

                # 更新 time_start 为最后一个 quote 的 time_close
                last_time_close = datetime.fromisoformat(quotes[-1]['time_close'].replace('Z', '+00:00'))  # 转换为 datetime 对象
                time_start = (last_time_close + timedelta(hours=1)).strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'  # 加一个小时并转换为指定格式
                print(f"Updated time_start to: {time_start}")  # 调试信息

                # 将数据保存到 CSV 文件
                if dataframes:
                    combined_df = pd.concat(dataframes, ignore_index=True)
                    combined_df.to_csv('ohlcv_data.csv', mode='a', header=not os.path.exists('ohlcv_data.csv'), index=False)  # 追加模式
                    print("Data appended to ohlcv_data.csv")
            else:
                print("No quotes found in the data. Increasing time_start by 1 hour.")  # 调试信息
                
                # 如果没有找到数据，增加 time_start 1 小时
                time_start = (datetime.fromisoformat(time_start.replace('Z', '+00:00')) + timedelta(hours=1)).strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'
        else:
            print("No data returned from fetch. Increasing time_start by 1 hour.")  # 调试信息
            
            # 如果没有返回数据，增加 time_start 1 小时
            time_start = (datetime.fromisoformat(time_start.replace('Z', '+00:00')) + timedelta(hours=1)).strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'

        # 获取当前时间戳并转换为 ISO 格式
        current_time = datetime.now(timezone.utc).strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'
        print(f"Current timestamp: {current_time}")  # 调试信息
            
        # 检查是否达到当前时间
        if time_start >= current_time:
            print("Reached the current timestamp, stopping the loop.")
            break

# 示例输入
contract_address = 'B4Vwozy1FGtp8SELXSXydWSzavPUGnJ77DURV2k4MhUV'
network_slug = 'Solana'
time_start = '2024-12-17T00:00:00.000Z'  # 确保使用正确的格式

main(contract_address, network_slug, time_start)

Starting fetch for time_start: 2024-12-17T00:00:00.000Z
Fetching data from URL: https://pro-api.coinmarketcap.com/v4/dex/pairs/ohlcv/historical?contract_address=B4Vwozy1FGtp8SELXSXydWSzavPUGnJ77DURV2k4MhUV&network_slug=Solana&time_period=hourly&interval=1h&count=50&time_start=2024-12-17T00:00:00.000Z
Request successful.
Data fetched successfully.
Number of quotes received: 38
Processed quote: {'convert_id': '2781', 'open': 0.027930835411092748, 'high': 0.027930835411092748, 'low': 0.027930835411092748, 'close': 0.027930835411092748, 'last_updated': None, 'volume': 267.11886844223744, 'time_open': '2024-12-17T12:00:00.000Z', 'time_close': '2024-12-17T12:59:59.999Z'}
Processed quote: {'convert_id': '2781', 'open': 0.027930835411092748, 'high': 0.07252444011416019, 'low': 0.03935857905176072, 'close': 0.055351392008524236, 'last_updated': None, 'volume': 187789687.00653037, 'time_open': '2024-12-17T13:00:00.000Z', 'time_close': '2024-12-17T13:59:59.999Z'}
Processed quote: {'convert_id': '